# Building Agentic RAG with Llamaindex
source: https://learn.deeplearning.ai/courses/building-agentic-rag-with-llamaindex/lesson/1/introduction

In [1]:
import os
import nest_asyncio

from dotenv import load_dotenv
from pathlib import Path
from typing import List, Optional

from llama_index.core import Settings, SimpleDirectoryReader, SummaryIndex, VectorStoreIndex
from llama_index.core.agent import FunctionCallingAgentWorker, AgentRunner
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.objects import ObjectIndex
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.vector_stores import MetadataFilters, FilterCondition
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI


from utils import display_text
from utils import filenames_in_directory

<span style="color: blue; font-size:30px;">Setup</span>

In [2]:
# necessary for running in Jupyter Notebook 
nest_asyncio.apply()

In [3]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
embed_model = OpenAIEmbedding(model="text-embedding-3-large")

Settings.llm = llm
Settings.embed_model = embed_model

<span style="color:blue; font-size:30px">LlamaIndex RAG functions</span>

In [5]:
def load_documents(file_path: str):
    """Load documents from the specified file path."""
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    return documents

def create_nodes(documents):
    """Create nodes from loaded documents."""
    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)
    return nodes

def create_vector_index(nodes):
    """Create a vector index from nodes."""
    vector_index = VectorStoreIndex(nodes)
    return vector_index

def vector_query(
    query: str, 
    vector_index: VectorStoreIndex, 
    page_numbers: Optional[List[str]] = None
) -> str:
    """Use to answer questions over a given paper.

    Useful if you have specific questions over the paper.
    Always leave page_numbers as None UNLESS there is a specific page you want to search for.

    Args:
        query (str): the string query to be embedded.
        page_numbers (Optional[List[str]]): Filter by set of pages. Leave as NONE 
            if we want to perform a vector search
            over all pages. Otherwise, filter by the set of specified pages.
    
    """
    
    page_numbers = page_numbers or []
    metadata_dicts = [{"key": "page_label", "value": p} for p in page_numbers]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(metadata_dicts, condition=FilterCondition.OR)
    )
    response = query_engine.query(query)
    return response

def create_vector_query_tool(name: str, vector_index: VectorStoreIndex):
    """Create a vector query tool."""
    return FunctionTool.from_defaults(
        name=f"vector_tool_{name}",
        fn=lambda query, page_numbers=None: vector_query(query, vector_index, page_numbers)
    )

def create_summary_index(nodes):
    """Create a summary index from nodes."""
    summary_index = SummaryIndex(nodes)
    return summary_index

def create_summary_tool(name: str, summary_index: SummaryIndex):
    """Create a summary tool."""
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
    )
    return QueryEngineTool.from_defaults(
        name=f"summary_tool_{name}",
        query_engine=summary_query_engine,
        description=(
            "Use ONLY IF you want to get a holistic summary related to {name} "
        ),
    )

<span style="color:blue; font-size:30px">Load Data into RAG</span>

In [9]:
file_path = "./input_docs_example"
papers = filenames_in_directory(file_path)
papers

['Foundation_Models_Enterprise_Workflows.pdf',
 'Ajua_Q2_2023_benchmark_report.txt',
 'metagpt.pdf']

In [10]:
def process_papers(papers, file_path):
    """
    Process a list of papers and create tools for each paper.

    Args:
        papers (list): A list of paper filenames.
        file_path (str): The path to the directory containing the papers.

    Returns:
        dict: A dictionary mapping each paper to its associated tools.
    """
    paper_to_tools_dict = {}
    
    for paper in papers:
        name = f"{Path(paper).stem}"
        print(f"Getting tools for paper: {paper}")
        
        # Assuming the following functions are defined elsewhere
        documents = load_documents(f"{file_path}/{Path(paper)}")
        nodes = create_nodes(documents)
        vector_index = create_vector_index(nodes)
        vector_query_tool = create_vector_query_tool(name, vector_index)
        summary_index = create_summary_index(nodes)
        summary_tool = create_summary_tool(name, summary_index)
        
        paper_to_tools_dict[paper] = [vector_query_tool, summary_tool]
    
    return paper_to_tools_dict

paper_to_tools_dict = process_papers(papers, file_path)


Getting tools for paper: Foundation_Models_Enterprise_Workflows.pdf
Getting tools for paper: Ajua_Q2_2023_benchmark_report.txt
Getting tools for paper: metagpt.pdf


In [11]:
# Flatten the list of tools from all papers into a single list `all_tools`
all_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]
print(f"number of tools: {len(all_tools)}") #check (should be 2 * number of papers)

number of tools: 6


In [12]:
# Define an "object" index and retriever over these tools
obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [14]:
# Initialize the FunctionCallingAgentWorker
# Verbose shows output of the agent's actions.

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=llm, 
    system_prompt=""" \
You are an agent designed to answer queries over a set of given papers.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    verbose=True
)
agent = AgentRunner(agent_worker)

<span style="color:blue; font-size:30px">Try it</span>

<span style="color:blue; font-size:20px">example</span>

In [15]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT include the Product Manager, Architect, Project Manager, Engineer, QA Engineer, CEO, individuals conducting experiments, designing modules, providing feedback, outlining methods, contributing to the write-up, team members assisting with experiments, comparisons, and creating illustrative figures, User Interface Designer, Data Scientist for data analysis, and System Agent. Each role has specific responsibilities and expertise tailored to different aspects of the collaborative framework within the MetaGPT development process.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "communication between agent roles in MetaGPT"}
=== Function Output ===
Communication between agent role

In [16]:
print(display_text(response.response))

In MetaGPT, the agent roles include the Product Manager, Architect, Project
Manager, Engineer, QA Engineer, CEO, individuals conducting experiments,
designing modules, providing feedback, outlining methods, contributing to the
write-up, team members assisting with experiments, comparisons, creating
illustrative figures, User Interface Designer, Data Scientist for data
analysis, and System Agent. Each role has specific responsibilities tailored to
different aspects of the collaborative framework.  Communication between these
agent roles in MetaGPT is structured and efficient. It is facilitated through
mechanisms such as role specialization, workflow management, and sharing
mechanisms like message pools and subscriptions. This structured approach
enhances communication efficiency, allowing agents to obtain directional
information from other roles and public information from the environment.
Agents publish structured messages in a shared message pool and can subscribe
to relevant messages

In [17]:
len(response.source_nodes)

68

In [18]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: input_docs_example/metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-06-24
last_modified_date: 2024-05-30

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
compl

In [19]:
print(response.source_nodes[1].get_content(metadata_mode="all"))

page_label: 2
file_name: metagpt.pdf
file_path: input_docs_example/metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-06-24
last_modified_date: 2024-05-30

Preprint
Figure 1: The software development SOPs between MetaGPT and real-world human teams.
In software engineering, SOPs promote collaboration among various roles. MetaGPT showcases
its ability to decompose complex tasks into specific actionable procedures assigned to various roles
(e.g., Product Manager, Architect, Engineer, etc.).
documents, design artifacts, flowcharts, and interface specifications. The use of intermediate struc-
tured outputs significantly increases the success rate of target code generation. Because it helps
maintain consistency in communication, minimizing ambiguities and errors during collaboration.
More graphically, in a company simulated by MetaGPT, all employees follow a strict and stream-
lined workflow, and all their handovers must comply with certain established standards. 

<span style="color:blue; font-size:20px">example</span>

In [20]:
response = agent.query(
    "According to Ajua, tell me about NPS and then how it is useful."
)

Added user message to memory: According to Ajua, tell me about NPS and then how it is useful.
=== Calling Function ===
Calling function: summary_tool_Ajua_Q2_2023_benchmark_report with args: {"input": "NPS"}
=== Function Output ===
NPS, or Net Promoter Score, is a metric utilized by businesses to assess customer loyalty and satisfaction by determining the likelihood of customers to recommend a company's products or services to others. This metric categorizes customers into Promoters, Passives, or Detractors based on their responses to a specific question, with Promoters being loyal advocates, Passives being satisfied but not enthusiastic, and Detractors being unhappy customers. The final NPS score is calculated by subtracting the percentage of Detractors from the percentage of Promoters, providing a valuable insight into customer perception and loyalty.
=== Calling Function ===
Calling function: summary_tool_Ajua_Q2_2023_benchmark_report with args: {"input": "usefulness of NPS"}
=== Fu

In [21]:
print(display_text(response.response))

Net Promoter Score (NPS) is a metric used by businesses to evaluate customer
loyalty and satisfaction by assessing the likelihood of customers to recommend
a company's products or services to others. It categorizes customers into
Promoters, Passives, or Detractors based on their responses to a specific
question. NPS is useful for businesses as it provides insights into customer
perception, loyalty, areas for improvement, customer satisfaction levels,
revenue trends, and market share predictions. Tracking NPS over time helps
companies assess performance, measure the effectiveness of customer service
efforts, and understand customer sentiment and loyalty.


In [22]:
len(response.source_nodes)

20

In [23]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

file_path: input_docs_example/Ajua_Q2_2023_benchmark_report.txt
file_name: Ajua_Q2_2023_benchmark_report.txt
file_type: text/plain
file_size: 34850
creation_date: 2024-06-24
last_modified_date: 2024-06-24

﻿AJUA


NPS Industry Benchmarks Q2 2023 
April 2023 – June 2023
________________


Table of Contents


- Background - 3
- Objectives - 4
- Methodology - 5
- Telcos - 7
- Mobile Money Lenders - 10
- Food & Beverage - 15
- Online Food Delivery Platforms - 19
- Retail/Modern Trade - 21
- Online Delivery Platforms - 25
- Pharmacies - 27
- Energy - 31
- Banking - 37
- Insurance - 41
- Healthcare - 44
________________


A remarkable customer experience is critical to the sustained growth of any business. A positive customer experience promotes loyalty, helps businesses retain customers, and encourages brand advocacy.
Today, customers have the power, not the sellers. Who gave them this power? Us — with help from the world wide web. The present customer has a plethora of options to choose fr

<span style="color:blue; font-size:20px">example</span>

In [24]:
#NOTE — Correct response: The NPS benchmark for Healthcare increased from 9 in Q2'22 to 20 in Q2'23.
response = agent.query(
    "How did the NPS benchmark for Healthcare change from Q2'22 to Q2'23? "
)

Added user message to memory: How did the NPS benchmark for Healthcare change from Q2'22 to Q2'23? 
=== Calling Function ===
Calling function: vector_tool_Ajua_Q2_2023_benchmark_report with args: {"query": "NPS benchmark Healthcare Q2'22 vs Q2'23"}
=== Function Output ===
NPS benchmark for Healthcare in Q2'22 was 9, while in Q2'23 it increased to 20.
=== LLM Response ===
The NPS benchmark for Healthcare increased from 9 in Q2'22 to 20 in Q2'23.


In [25]:
#NOTE — Correct response: The NPS benchmark for TELCO decreased from 29 in Q2'22 to 24 in Q2'23 according to the Ajua report.
response = agent.chat(
    "According to the Ajua report, how did the NPS benchmark for TELCO change from Q2'22 to Q2'23? "
)

Added user message to memory: According to the Ajua report, how did the NPS benchmark for TELCO change from Q2'22 to Q2'23? 
=== Calling Function ===
Calling function: vector_tool_Ajua_Q2_2023_benchmark_report with args: {"query": "NPS benchmark TELCO Q2'22 vs Q2'23"}
=== Function Output ===
Telco's NPS benchmark in Q2'22 was 29, while in Q2'23 it decreased to 24.
=== LLM Response ===
The NPS benchmark for TELCO decreased from 29 in Q2'22 to 24 in Q2'23 according to the Ajua report.


<span style="color:blue; font-size:20px">example</span>

In [29]:
#NOTE: Pharmacy is the correct answer. But needed to ask a carefully crafted question to get the answer.
response = agent.chat(
    "Which industry (not company) showed the best performance in the second quarter of 2023 according to the Ajua report? "
)

Added user message to memory: Which industry (not company) showed the best performance in the second quarter of 2023 according to the Ajua report? 
=== Calling Function ===
Calling function: vector_tool_Ajua_Q2_2023_benchmark_report with args: {"query": "best industry performance Q2'23"}
=== Function Output ===
The Pharmacy Industry demonstrated the best performance in Q2'23 with an Overall Net Promoter Score (NPS) of 38.
=== LLM Response ===
According to the Ajua report, the Pharmacy Industry demonstrated the best performance in the second quarter of 2023 with an Overall Net Promoter Score (NPS) of 38.


<span style="color:blue; font-size:20px">example</span>

In [30]:
# NOTE — The answer should come from the metaGPT paper.
response = agent.chat(
    "What are the roots of automatic programming?"
)

Added user message to memory: What are the roots of automatic programming?
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "Roots of automatic programming"}
=== Function Output ===
The roots of automatic programming can be traced back to the early developments in the field, with initial ideas of recursive self-improvement informally proposed in 1965 and concretely implemented starting from 1987. Over time, concepts of mathematically optimal self-referential self-improvers were introduced, leading to advancements in learning algorithms and meta-learning. Recent work has leveraged Large Language Models (LLMs) to enhance performance on various tasks through recursive prompt modifications. Additionally, there have been explorations into self-referential mechanisms that adapt constraint prompts based on project experiences, aiming to improve individual agents within a system continuously.
=== LLM Response ===
The roots of automatic programming can be tra

In [31]:
print(display_text(response.response))

The roots of automatic programming can be traced back to early developments in
the field, with initial ideas of recursive self-improvement proposed informally
in 1965 and concretely implemented starting from 1987. Concepts of
mathematically optimal self-referential self-improvers have been introduced,
leading to advancements in learning algorithms and meta-learning. Recent work
has utilized Large Language Models (LLMs) to enhance performance on various
tasks through recursive prompt modifications. There have also been explorations
into self-referential mechanisms that adapt constraint prompts based on project
experiences to continuously improve individual agents within a system.


In [32]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: input_docs_example/metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-06-24
last_modified_date: 2024-05-30

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
compl

<span style="color:blue; font-size:20px">example</span>

In [33]:
response = agent.chat(
    "Summarize the paper discussing Multimodal Foundation Models and Enterprise Workflows."
)

Added user message to memory: Summarize the paper discussing Multimodal Foundation Models and Enterprise Workflows.
=== Calling Function ===
Calling function: summary_tool_Foundation_Models_Enterprise_Workflows with args: {"input": "Multimodal Foundation Models and Enterprise Workflows"}
=== Function Output ===
Multimodal foundation models play a crucial role in enhancing enterprise workflows by assisting in tasks such as documentation, knowledge transfer, and workflow improvement. These models have shown capabilities in generating accurate documentation, refining standard operating procedures (SOPs), and improving workflow quality without human intervention. However, challenges persist in achieving enterprise-level accuracy, particularly in areas like low-level error correction and accurately ranking SOPs based on quality as perceived by human annotators. The introduction of benchmarks like WONDERBREAD aims to encourage the development of more "human-centered" AI tools for enterprise 

<span style="color:blue; font-size:20px">example</span>

In [34]:
response = agent.chat(
    "Compare and contrast (1) the paper discussing Multimodal Foundation Models and Enterprise Workflows and (2) the metagpt paper."
)

Added user message to memory: Compare and contrast (1) the paper discussing Multimodal Foundation Models and Enterprise Workflows and (2) the metagpt paper.
=== Calling Function ===
Calling function: summary_tool_Foundation_Models_Enterprise_Workflows with args: {"input": "Multimodal Foundation Models and Enterprise Workflows"}
=== Function Output ===
Multimodal Foundation Models are being utilized in enterprise workflows to enhance automation, decision-making processes, and overall efficiency within organizations. These models are trained on vast datasets and combine natural language understanding with visual processing capabilities to tackle various business process management (BPM) tasks. The models are evaluated based on their performance in generating Standard Operating Procedures (SOPs), segmenting demonstrations, answering questions, validating workflows, and improving SOP quality. While they have shown potential in improving the quality of their own SOPs and adapting to changin

In [35]:
print(display_text(response.response))

The paper discussing Multimodal Foundation Models and Enterprise Workflows
focuses on the utilization of multimodal models in enterprise workflows to
enhance automation, decision-making processes, and overall efficiency within
organizations. These models combine natural language understanding with visual
processing capabilities to address various business process management tasks,
such as generating Standard Operating Procedures (SOPs), segmenting
demonstrations, answering questions, validating workflows, and improving SOP
quality. While these models have shown potential in improving SOP quality and
adapting to changing workflows, challenges exist in aligning model judgments
with human judgments, particularly in tasks like SOP ranking. The integration
of multimodal models in enterprise workflows offers opportunities to enhance
efficiency, accuracy, and scalability in diverse operational contexts.  On the
other hand, the metagpt paper compares Multimodal Foundation Models and
Enterprise

<span style="color:blue; font-size:20px">example</span>

In [36]:
response = agent.chat(
    "According to the Ajua report, what were the main factors influencing custoner experience in dining?"
)

Added user message to memory: According to the Ajua report, what were the main factors influencing custoner experience in dining?
=== Calling Function ===
Calling function: vector_tool_Ajua_Q2_2023_benchmark_report with args: {"query": "factors influencing customer experience dining"}
=== Function Output ===
Customer service, food and drink quality, and the ambiance of the outlet are the primary factors influencing customer experience dining.
=== LLM Response ===
According to the Ajua report, the main factors influencing customer experience in dining include customer service, food and drink quality, and the ambiance of the outlet.


In [38]:
# NOTE: Good answer:  According to the Ajua report, the main factors influencing customer experience in dining include 
# customer service, food and drink quality, and the ambiance ofthe outlet.
print(display_text(response.response))

According to the Ajua report, the main factors influencing customer experience
in dining include customer service, food and drink quality, and the ambiance of
the outlet.


<span style="color:blue; font-size:20px">To do. Looking for a way to clear chat/memory. </span>

In [39]:
# not certain that this working
agent.reset()

In [40]:
agent.chat("Acording to the Ajua report, how did the NPS benchmark for Insurance change from Q2'22 to Q2'23?")

Added user message to memory: Acording to the Ajua report, how did the NPS benchmark for Insurance change from Q2'22 to Q2'23?
=== Calling Function ===
Calling function: vector_tool_Ajua_Q2_2023_benchmark_report with args: {"query": "NPS benchmark for Insurance", "page_numbers": [1]}
=== Function Output ===
Empty Response
=== LLM Response ===
I couldn't find specific information on how the NPS benchmark for Insurance changed from Q2'22 to Q2'23 in the Ajua report. Would you like me to provide a summary of the report to see if there are any related insights?


AgentChatResponse(response="I couldn't find specific information on how the NPS benchmark for Insurance changed from Q2'22 to Q2'23 in the Ajua report. Would you like me to provide a summary of the report to see if there are any related insights?", sources=[ToolOutput(content='Empty Response', tool_name='vector_tool_Ajua_Q2_2023_benchmark_report', raw_input={'args': (), 'kwargs': {'query': 'NPS benchmark for Insurance', 'page_numbers': [1]}}, raw_output=Response(response='Empty Response', source_nodes=[], metadata=None), is_error=False)], source_nodes=[], is_dummy_stream=False)

In [41]:
response = agent.chat(
    "Remind me, what are we discussing?"
)

Added user message to memory: Remind me, what are we discussing?
=== LLM Response ===
We were discussing how the NPS benchmark for Insurance changed from Q2'22 to Q2'23 according to the Ajua report. Would you like me to provide a summary of the report to see if there are any related insights?


In [42]:
# not certain that this working
agent.reset()

In [43]:
response = agent.chat(
    "Remind me, what are we discussing?"
)

Added user message to memory: Remind me, what are we discussing?
=== LLM Response ===
We are discussing enterprise workflows and their foundation models. How can I assist you further with this topic?
